In [ ]:
print("Is the kernel alive?")
print("Yes, the kernel is alive")

In [2]:
# First lets install smolagents
# %pip install -q -U smolagents
# %pip install -q -U ipywidgets

In [ ]:
# Login to Hugging Face Either use the notebook_login, command line or load the .env file
# from huggingface_hub import notebook_login
# notebook_login()

# Load the .env file, the following are loaded from the .env file
# 1. HF_TOKEN

from dotenv import load_dotenv
load_dotenv()


In [ ]:
#@title Connecting to LM Studio
import requests

# Define LM Studio endpoint
LM_STUDIO_URL = "http://127.0.0.1:1234"

# Test that LM Studio is working
try:
    response = requests.get(f"{LM_STUDIO_URL}/v1/models")
    if response.status_code == 200:
        print("✅ LM Studio is running!")
        print("Available models:", response.json())
    else:
        print("⚠️ Unable to reach LM Studio. Make sure it's running.")
except requests.exceptions.ConnectionError:
    print("❌ LM Studio is not running. Start it and try again.")

In [ ]:
prompt = "What is the the best fruit? Choose one."

llm_model = "deepseek-r1-distill-qwen-14b"
max_tokens = 4096
temperature = 0.6
# Define the request payload for LM Studio
payload = {
    "model": llm_model,
    "promt": prompt,
    "messages": [
        {"role": "user", "content": prompt}
        ],
    "max_tokens": max_tokens,
    "temperature": temperature
}

# Send the request to LM Studio
response = requests.post(f"{LM_STUDIO_URL}/v1/chat/completions", json=payload)

# Parse and print the response

if response.status_code == 200:
    result = response.json()
    print("🤖 LM Studio:", result["choices"][0]["message"]["content"])  # Fixed key
else:
    print("❌ Something went wrong:", response.text)

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel

agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=HfApiModel())

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")

In [ ]:
# !pip install smolagents[transformers]
from smolagents import CodeAgent, TransformersModel

model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = TransformersModel(model_id=model_id)
agent = CodeAgent(tools=[], model=model, add_base_tools=True)

agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

In [ ]:
# %pip install smolagents[litellm]
from smolagents import CodeAgent, LiteLLMModel
prompt = "What should I wear? Blue o red?"

local_model = LiteLLMModel(
    model_id="lm_studio/deepseek-r1-distill-qwen-14b", # This model is a bit weak for agentic behaviours though
    api_base="http://127.0.0.1:1234/v1", # replace with 127.0.0.1:11434 or remote open-ai compatible server if necessary
    api_key="lm-studio-api-key", # replace with API key if necessary
    num_ctx=8192, # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
    
)

local_agent = CodeAgent(
    tools=[], 
    model=local_model, 
    add_base_tools=True)

local_agent.run(
    prompt
)

In [ ]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."
    
agent = CodeAgent(
    tools=[suggest_menu],
    model=HfApiModel(),
)

agent.run("Prepare a formal menu for today's dinner.")

In [20]:
from smolagents import CodeAgent, tool

local_model = LiteLLMModel(
    model_id="lm_studio/deepseek-r1-distill-qwen-14b",
    api_base="http://127.0.0.1:1234/v1", # replace with 127.0.0.1:11434 or remote open-ai compatible server if necessary
    api_key="lm-studio-api-key", # replace with API key if necessary
    num_ctx=8192, # ollama default is 2048 which will fail horribly. 8192 works for easy tasks, more is better. Check https://huggingface.co/spaces/NyxKrage/LLM-Model-VRAM-Calculator to calculate how much VRAM this will need for the selected model.
)

@tool
def blinds_control(blinds_state: bool) -> str:
    """
    Controls the state of the house blinds.

    Args:
        blinds_state: True to open the blinds, False to close the blinds.

    Returns:
        str: The state of the blinds.
    """
    if blinds_state:
        return "Blinds are open"
    else:return "Blinds are closed"

local_agent = CodeAgent(
    tools=[blinds_control],
    model=local_model,
)

# local_agent.run("Close the blinds please. I think it's too bright outside.")



In [ ]:
from smolagents import CodeAgent, HfApiModel
import numpy as np
import time
import datetime

agent = CodeAgent(tools=[], model=HfApiModel(), additional_authorized_imports=['datetime'])

agent.run(
    """
    Alfred needs to prepare for the party. Here are the tasks:
    1. Prepare the drinks - 30 minutes
    2. Decorate the mansion - 60 minutes
    3. Set up the menu - 45 minutes
    4. Prepare the music and playlist - 45 minutes

    If we start right now, at what time will the party be ready?
    """
)

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool, tool, Tool
import datetime

@tool
def check_time() -> str:
    """Checks the current time.
    
    Args:
        None
    Returns:
        str: The current time in the format HH:MM:SS.
    """
    return datetime.datetime.now().strftime("%H:%M:%S")

@tool
def house_windows_control(windows_state: bool) -> str:
    """Controls the state of the house windows.

    Args:
        windows_state: True to open the windows, False to close the windows.

    Returns:
        str: The state of the windows.
    """
    if windows_state:
        return "Windows are open"
    else:
        return "Windows are closed"

@tool
def blinds_control(blinds_state: bool) -> str:
    """
    Controls the state of the house blinds.

    Args:
        blinds_state: True to open the blinds, False to close the blinds.

    Returns:
        str: The state of the blinds.
    """
    if blinds_state:
        return "Blinds are open"
    else:return "Blinds are closed"

local_agent = CodeAgent(
    tools=[blinds_control, house_windows_control, DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=local_model,
    add_base_tools=True,
    additional_authorized_imports=['datetime']
)
agent = CodeAgent(
    tools=[blinds_control, house_windows_control, DuckDuckGoSearchTool(), VisitWebpageTool(), check_time],
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2,
    add_base_tools=True,
    additional_authorized_imports=['datetime']
)
prompt = "Is it raining? If it is, please close the windows and blinds. What time is it by the way?"

local_agent.run(prompt)
# agent.run(prompt)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Is it raining? If it is, please close the windows and blinds. What time is it by the way?                       │
│                                                                                                                 │
╰─ LiteLLMModel - lm_studio/deepseek-r1-distill-qwen-14b ─────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Exception while exporting Span.
Traceback (most recent call last):
  File "c:\Users\angel\Documents\projects\my_little_local_agents\.venv\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\angel\Documents\projects\my_little_local_agents\.venv\Lib\site-packages\urllib3\connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "c:\Users\angel\Documents\projects\my_little_local_agents\.venv\Lib\site-packages\urllib3\connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\http\client.py", line 1395, in getresponse
    response.begin()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\Lib\htt

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Ask for location                                                                                       
  location = input("Please enter your city or country: ")                                                          
  print(f"Checking weather in {location}...")                                                                      
                                                                                                                   
  # Step 2: Search if it's raining                                                                                 
  is_raining = False                                                                                               
  search_results = web_search(query=f"is it raining in {location}")                                                
  if "yes" in search_results.lower():                                                                              
      is_raining = True                                                                                            
                                                                                                                   
  # Step 3: Close windows and blinds if raining                                                                    
  if is_raining:                                                                                                   
      house_windows_control(windows_state=False)                                                                   
      print("Windows closed due to rain.")                                                                         
      blinds_control(blinds_state=False)                                                                           
      print("Blinds closed due to rain.")                                                                          
                                                                                                                   
  # Step 4: Get current time                                                                                       
  import datetime                                                                                                  
  current_time = datetime.datetime.now()                                                                           
  print(f"Current Time: {current_time}")                                                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'location = input("Please enter your city or country: ")' due to: InterpreterError: 
It is not permitted to evaluate other functions than the provided tools or functions defined/imported in previous 
code (tried to execute input).

...

Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}
Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


Execution logs:
Checking weather in London...
Web Search Results: ## Search Results

[London - BBC Weather](https://www.bbc.com/weather/2643743)
14-day weather forecast for London.

[London, London, United Kingdom Current Weather | 
AccuWeather](https://www.accuweather.com/en/gb/london/ec4a-2/current-weather/328328)
Current weather in London, London, United Kingdom. Check current conditions in London, London, United Kingdom with 
radar, hourly, and more.

[London local weather (live): today, hourly 
weather](https://www.weather25.com/europe/united-kingdom/london?page=today)
The temperature in London today in the early morning is 3 ° C. If you take into account factors such as wind, 
humidity and other weather conditions, the temperatures can feel like 2 ° C. The chance of rain in London in the 
morning is 0%, and the wind will blow at 4 km/h.

[London (Greater London) weather - Met Office](https://weather.metoffice.gov.uk/forecast/gcpvj0v07)
London 7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV

[Will It Rain In London? - Rainfall Forecast - 
Netweather.tv](https://www.netweather.tv/weather-forecasts/uk/will-it/rain/18351~London)
About Will It Rain. Will it rain is updated 4 times a day using the very latest data, so check back regularly to 
stay upto date. The figures you see are the total amounts of rainfall forecast for each day and the risk of rain 
falling, with the graphic giving an indication of how heavy or persistant you can expect that rain to be.

[London, England, United Kingdom Weather - The Weather 
Channel](https://weather.com/weather/hourbyhour/l/London+England+United+Kingdom?canonicalCityId=805c6df1c2518951d4d
ffa28cf6e483358644c3131109dff6c9a8a2c265fbc67)
Hourly Local Weather Forecast, weather conditions, precipitation, dew point, humidity, wind from Weather.com and 
The Weather Channel

[Weather Today for London, London, United Kingdom | 
AccuWeather](https://www.accuweather.com/en/gb/london/ec4a-2/weather-today/328328)
Everything you need to know about today's weather in London, London, United Kingdom. High/Low, Precipitation 
Chances, Sunrise/Sunset, and today's Temperature History.

[Today's Weather in London - Hourly Forecast and 
Conditions](https://www.easeweather.com/europe/united-kingdom/england/greater-london/london/today)
Get the latest hourly weather updates for London today. Detailed forecast including temperature, wind, rain, snow, 
and UV index. Stay informed about today's weather conditions in London.

[Weather in London, England, United Kingdom - timeanddate.com](https://www.timeanddate.com/weather/uk/london)
Current weather in London and forecast for today, tomorrow, and next 14 days

[Is It Going To Rain Today in London, UK](https://isitgoingtoraintoday.com/london-uk/)
Find out is it going to rain today in london, uk. Today, the weather forecast predicts overcast clouds, with 100% 
cloud coverage.There's 20 probability of precipitation, and if rain occurs, it's expected to measure 0.41 
millimeters.The temperature is forecasted to range between 11°C and 20°C.Additionally, the wind speed is expected 
to be around 12 km/h.
It is not raining.
No need to close windows and blinds as it is not raining.
Current Time: 2025-03-04 19:15:11.489442
Checking weather in London...
Web Search Results: ## Search Results

[London - BBC Weather](https://www.bbc.com/weather/2643743)
14-day weather forecast for London.

[London, London, United Kingdom Current Weather | 
AccuWeather](https://www.accuweather.com/en/gb/london/ec4a-2/current-weather/328328)
Current weather in London, London, United Kingdom. Check current conditions in London, London, United Kingdom with 
radar, hourly, and more.

[London local weather (live): today, hourly 
weather](https://www.weather25.com/europe/united-kingdom/london?page=today)
The temperature in London today in the early morning is 3 ° C. If you take into account factors such as wind, 
humidity and other weather conditio

Failed to export batch code: 401, reason: {"message":"Invalid credentials. Confirm that you've configured the correct host."}


[Step 2: Duration 1099.21 seconds| Input tokens: 9,151 | Output tokens: 2,948]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
local_agent.push_to_hub('angelUndeveloped/windows_blinds_agent')

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, Tool, tool, VisitWebpageTool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion: The type of occasion for the party.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.
    
    Args:
        query: A search term for finding catering services.
    """
    # Example list of catering services and their ratings
    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }
    
    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)
    
    return best_service

class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description = """
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""
    
    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }
    
    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }
        
        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(), 
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool()
    ], 
    model=HfApiModel(),
    max_steps=10,
    verbosity_level=2
)

agent.run("Give me the best playlist for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme")

In [ ]:
%pip install -q opentelemetry-sdk opentelemetry-exporter-otlp openinference-instrumentation-smolagents

In [ ]:
import os
import base64
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Safely get keys from environment variables
LANGFUSE_PUBLIC_KEY = os.getenv('LANGFUSE_PUBLIC_KEY')
LANGFUSE_SECRET_KEY = os.getenv('LANGFUSE_SECRET_KEY')

# Verify keys are loaded
if not LANGFUSE_PUBLIC_KEY or not LANGFUSE_SECRET_KEY:
    raise ValueError("Langfuse keys not found in .env file")

# Create auth token
LANGFUSE_AUTH = base64.b64encode(
    f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()
).decode()

# Set up OTEL environment
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

In [9]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)


In [ ]:
from smolagents import CodeAgent, HfApiModel

hf_agent = CodeAgent(
    tools=[],
    model=HfApiModel(),
    max_steps=10,
)